In [1]:
from parser import ConsumptionDataParser
import pandas as pd


In [2]:
LOCAL_TIMEZONE = "Europe/Vienna"
FILE = "/Users/matsschneider/Documents/Privat/Ideas/Strompreis Analyse/VIERTELSTUNDENWERTE-20250724.csv"

In [3]:
df = pd.read_csv(FILE, sep=";", decimal=",", encoding="utf-8", dayfirst=True)
consumption_col = next((col for col in df.columns if "Verbrauch [kWh]" in col), None)

if not consumption_col or "Datum" not in df.columns or "Zeit von" not in df.columns:
    raise ValueError("Default format columns not found.")
    
df = df[["Datum", "Zeit von", consumption_col]].copy()#.dropna()
df.columns = ["date_str", "time_str", "consumption_kwh"]

# Manually combine date and time for robust parsing
df["timestamp_local"] = pd.to_datetime(df["date_str"] + " " + df["time_str"], dayfirst=True)
df["timestamp"] = df["timestamp_local"].dt.tz_localize(LOCAL_TIMEZONE, ambiguous="infer").dt.tz_convert("UTC")

In [4]:
grp = df.groupby(pd.Grouper(key="timestamp", freq="MS"))["consumption_kwh"].sum()
grp[grp.index >= "01.01.2025"]

timestamp
2025-01-01 00:00:00+00:00    108.592
2025-02-01 00:00:00+00:00    108.792
2025-03-01 00:00:00+00:00    124.774
2025-04-01 00:00:00+00:00    107.984
2025-05-01 00:00:00+00:00    102.721
2025-06-01 00:00:00+00:00    100.256
2025-07-01 00:00:00+00:00     58.867
Freq: MS, Name: consumption_kwh, dtype: float64